# Partie 2 : Transmission et réception des données

Dans cette partie, vous allez transmettre vos données vers les serveurs applicatifs en utilisant la
technologie LoRaWAN. Vous utiliserez un module LoRa Mote de microchip.  
Documentation du module :
https://ww1.microchip.com/downloads/en/DeviceDoc/RN2483-LoRa-Technology-Module-Command-Reference-User-Guide-DS40001784G.pdf

Ressources externes :
https://github.com/CampusIoT/tutorial/blob/master/rn2483/README.md

## 1. Connexion au module

Afin de pouvoir établir une connexion avec la LoRa Mote, nous allons importer serial.  
Pour installer serial :  
`pip install pyserial`

Il faut que vous trouviez le port tty utilisé par le module.  

Vous pouvez utiliser la commande :
`ls /dev |grep tty`

In [3]:
#Import libraries
!pip install pyserial
import time
import logging
import serial

Looking in indexes: https://pypi.org/simple, https://www.piwheels.org/simple

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
#Configuration
PORT     = '/dev/ttyACM0'
BAUDRATE = 57600
logger= logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Test")

DEBUG:root:Test


Afin de faciliter la connexion au module, nous vous proposons la fonction ci-dessous.


In [5]:
def setup_serial(port:str=PORT,baudrate:int=BAUDRATE,bytesize:int=serial.EIGHTBITS,
                        parity:str=serial.PARITY_NONE,stopbits:int=serial.STOPBITS_ONE,
                        dtr:int=False):
    """
    Function to setup my serial connection
    Params:
        port:str        : Port used for my connection, default value PORT
        baudrate:int    : Baudrate, default value BAUDRATE
        bytesize:int    : bytesize, default value serial.EIGHTBITS
        parity:str      : Bit parity, default value serial.PARITY_NONE
        stopbits:int    : Stop bits, default value serial.STOPBITS_ONE
        dtr:bool        : Data Terminal Ready, default value False
    Returns :
        sp:Serial.Serial: A serial connection
    """
    try:
        sp = serial.Serial()
        sp.port = port
        sp.baudrate = baudrate
        sp.bytesize = bytesize
        sp.parity = parity
        sp.stopbits = stopbits
        sp.dtr=dtr
        sp.open()
        return sp
    except (ValueError,serial.SerialException) as exception:
        logging.critical("Could not open the serial connection.")
        raise exception

In [6]:
#Test de connexion
module = setup_serial()

## 2. Paramétrage du module

Dans cette partie, il est attendu que l'élève recherche les différentes commandes à envoyer au module pour définir les fonctions suivantes.

La fonction send() est fournie.

In [7]:
def send(sp:serial.Serial,data:str):
    """
    Send data through the serial connection
        Param:
            sp:serial.Serial : serial.Serial object used for the RN2485
            data:str : Data to encode and send
        Returns :
            decoded_response:str: Returns a response if got one
    """
    #Encode data and send it through the serial connection
    data_to_send = (data.rstrip()+"\x0d\x0a").encode()
    sp.write(data_to_send)
    time.sleep(0.2)

    #Wait for a response
    rdata=sp.readline()
    while not rdata:
        rdata = sp.readline()

    #Decode response and send it
    decoded_response = rdata.strip().decode()
    logging.debug("Decoded response : %s",decoded_response)
    return decoded_response

In [8]:
def reset_module(sp):
    """cnf
    Reset the module
    sys reset : This command resets and restarts the RN2483 module; stored LoRaWAN protocol
    settings will be loaded automatically upon reboot.
        Param:
        Returns :
            response:str : Response from the module
            response from the module : RN2483 X.Y.Z MMM DD YYYY HH:MM:SS, where X.Y.Z is firmware
            version, MMM is month, DD is day, YYYY is year, HH:MM:SS is hour,
            minutes, seconds (format: [HW] [FW] [Date] [Time]). [Date] and [Time] refer
            to the release of the firmware.
            
    """
    #Send the command to reset the module
    command = "sys factoryRESET"
    response = send(sp,command)
    return response

In [9]:
def set_appkey(sp,appkey:str):
    """
    Set the APPKEY
    mac set appkey <appKey> : 
        - This command sets the application key for the module. The application key is used tocnf
          derive the security credentials for communication during over-the-air activation.
        Param:
            <appKey>: 16-byte hexadecimal number representing the application key
        Returns :
            Response from module: 
                    - ok if key is valid
                    - invalid_param if key is not valid
    """
    #Send the command to set the appkey
    command = "mac set appkey "+appkey
    response = send(sp,command)
    return response

In [10]:
def set_joineui(sp,joineui:str):
    """
    Set the JOINEUI
    mac set appeui <appEUI>
        This command sets the 
        Param:
                ??
        Returns :
            Response: ok if address is valid
            invalid_param if address is not valid
    """
    #Send the command to set the joineui
    command = "mac set appeui " + joineui
    response = send(sp,command)
    return response

In [11]:
def set_datarate(sp,spreading_factor:int):
    """
    Set the datarate
    mac set dr <dataRate>
    This command sets the data rate to be used for the next transmission. 
        Param:
            <dataRate>: decimal number representing the data rate, from 0 and 7, 
            but within the limits of the data rate range for the defined channels.
        Returns :
            Response: ok if data rate is valid or invalid_param if data rate is not valid
    """
    datarate = "DR0" #Find the relation between SF and DR
    #Send the command to set the datarate
    command = "mac set dr " + str(spreading_factor)
    response = send(sp,command)
    return response

In [12]:
def set_deveui(sp,deveui:str):
    """
    Set the DEVEUI
    mac set deveui <devEUI>
        This command sets the globally unique device identifier for the module. The identifier
        must be set by the host MCU. The module contains a pre-programmed unique EUI and
        can be retrieved using the sys get hweui command (see Section 2.3.6.4) or user
        provided EUI can be configured using the mac set deveui command.
        Param:
            <devEUI>: 8-byte hexadecimal number representing the device EUI
        Returns :
            Response: ok if address is valid ou invalid_param if address is not validcnf
    """
    #Send the command to set the deveui
    command = "mac set deveui " + deveui
    response = send(sp,command)
    return response

In [13]:
def save_config(sp):
    """
    The mac save command must be issued after configuration parameters have been
    appropriately entered from the mac set <cmd> commands. This command will save
    LoRaWAN Class A protocol configuration parameters to the user EEPROM. When the
    next sys reset command is issued, the LoRaWAN Class A protocol configuration will
    be initialized with the last saved parameters.
        Param:
        Returns :
            Response: ok
    """
    #Send the command to save the current config
    command = "mac save"
    response = send(sp,command)
    return response

### Le data rate, c'est quoi ?  

Le Data Rate (DR) c'est le débit de données. Comme vous l'avez vu en cours et en TP, le facteur d'étalement (Spreading Factor, SF) à un lien direct avec ce dernier.  
Pour pouvoir choisir un facteur d'étalement avec ce module, vous allez devoir choisir la configuration associée au niveau de DR souhaité.  


Extrait de LoRaWAN™ Specification V1.0.2

Data Rate | Spreading Factor | Bandwidth | bits/s
---------|-------------------|----------|--------------------
DR0      | SF12             | 125 kHz   | 250
DR1      | SF11             | 125 kHz   | 440
DR2      | SF10             | 125 kHz   | 980
DR3      | SF9              | 125 kHz   | 1760
DR4      | SF8              | 125 kHz   | 3125
DR5      | SF7              | 125 kHz   | 5470
DR6      | SF7              | 250 kHz   | 11000
DR7      | FSK              | 50 kbps    | 50000

*(Note : Ici, vous utiliserez les configuration de data rate comprises entre 0 et 5)*

#### Rappel - Spreading Factor

Cette vidéo explique la modulation utilisée par LoRa : 
https://www.youtube.com/watch?v=dxYY097QNs0

LoRa utilise la modulation CSS (Chirp Spread Spectrum), où les *chirps* (ou symboles) vont transporter les données.  
Le Spreading Factor - ou facteur d'étalement - contrôle l'étalement du chirp dans le temps.  
Plus le facteur d'étalement est elevé, plus le *chirp* est étendu dans le temps; le message reste alors plus longtemps dans l'air (le Time On Air augmente).

Quand le SF est faible, on envoie des messages plus rapidement. On augmente donc le débit au détriment de la portée; en envoyant des signaux plus courts ces derniers sont plus vulnérables aux bruits et interférences.  
Quand le SF est élevé, on envoie des messages plus lentement. Le débit baisse, mais la portée augmente; en envoyant des signaux plus longs ces derniers sont moins vulnérables aux bruits et interférences. Les erreurs sont également plus facilement corrigées grace a la redondance du *chirp*.

Dans cette fonction, vous allez ré-utiliser les fonctions définies plus haut pour configurer le module.


In [14]:
def config_module(sp,appkey:str,joineui:str,deveui:str,spreading_factor:int):
    """
    Configurate the module
        Param:
            spreading_factor:int : ???????????
            joineui:str          : ???????????
            deveui:str           : ???????????
            spreading_factor:int : ???????????
        Returns :
            bool: True if joined, False if not
    """
    logging.info("Resetting device")
    #response = send(sp, "sys reset")
    logging.debug("Response : %s",reset_module(sp))
    
    logging.info("Setting APPKEY : %s",appkey)
    #response = send(sp, f"mac set appkey {appkey}")
    logging.debug("Response : %s",set_appkey(sp, appkey))
    
    logging.info("Setting JOINEUI : %s",joineui)
    #response = send(sp, f"mac set joineui {joineui}")
    logging.debug("Response : %s",set_joineui(sp,joineui))
    
    logging.info("Setting DEVEUI : %s",deveui)
    #response = send(sp, f"mac set deveui {deveui}")
    logging.debug("Response : %s",set_deveui(sp,deveui))
    
    logging.info("Setting the data-rate, spreading_factor = %s",spreading_factor)
    #response = send(sp, f"mac set dr {spreading_factor}")
    logging.debug("Response : %s",set_datarate(sp,spreading_factor))
    
    logging.info("Saving mac settings")
    #response = send(sp, "mac save")
    logging.debug("Response : %s",save_config(sp))
    
    #Here we disable the duty cycle limit on the module to avoid errors
    for channel in range(0,3):
        #Change duty cycle
        logging.info("Setting channel %s duty cycle to  1.00",channel)
        response = send(sp,f"mac set ch dcycle {channel} 1")
        logging.info("Set %s to dcycle response : %s",channel,response)
        #Channel status to on
        logging.info("Setting channel %s to on",channel)
        response = send(sp,f"mac set ch status {channel} on")
        logging.info("Set %s on response : %s",channel,response)
    
    
    #Now we join the network, this part is given
    joining=False
    while not joining:
        logging.info("Preparing to join the network")
        response = send(sp,"mac join otaa")
        logging.info("Mac join otaa response : %s",response)
        if "ok" in response:
            joining=True
        time.sleep(2)
        
    logging.info("Wating to get the accepted response")
    time.sleep(2) #Wait for accepted response
    ret = sp.readline()
    while not ret:
        ret = sp.readline()
    response = ret.strip().decode()
    logging.info("Status of the join request : %s",response)
    if not "accepted" in response:
        return False
    return True

In [15]:
# Test de la fonction
response = config_module(module,"0123456789ABCDEF0123456789ABCDEF","DEAD25DEAD25DEAD","DEADDEAD00090007", 4)
logging.info("Did we join the network ? %s",response)

INFO:root:Resetting device
DEBUG:root:Decoded response : RN2483 1.0.1 Dec 15 2015 09:38:09
DEBUG:root:Response : RN2483 1.0.1 Dec 15 2015 09:38:09
INFO:root:Setting APPKEY : 0123456789ABCDEF0123456789ABCDEF
DEBUG:root:Decoded response : ok
DEBUG:root:Response : ok
INFO:root:Setting JOINEUI : DEAD25DEAD25DEAD
DEBUG:root:Decoded response : ok
DEBUG:root:Response : ok
INFO:root:Setting DEVEUI : DEADDEAD00090007
DEBUG:root:Decoded response : ok
DEBUG:root:Response : ok
INFO:root:Setting the data-rate, spreading_factor = 4
DEBUG:root:Decoded response : ok
DEBUG:root:Response : ok
INFO:root:Saving mac settings
DEBUG:root:Decoded response : ok
DEBUG:root:Response : ok
INFO:root:Setting channel 0 duty cycle to  1.00
DEBUG:root:Decoded response : ok
INFO:root:Set 0 to dcycle response : ok
INFO:root:Setting channel 0 to on
DEBUG:root:Decoded response : ok
INFO:root:Set 0 on response : ok
INFO:root:Setting channel 1 duty cycle to  1.00
DEBUG:root:Decoded response : ok
INFO:root:Set 1 to dcycle re

In [16]:
# Verification bon SF 
logging.info("Getting SF : %s")
response = send(module, f"radio get sf ")
logging.debug("Response : %s",response)

INFO:root:Getting SF : %s
DEBUG:root:Decoded response : sf12
DEBUG:root:Response : sf12


# 3. Envoi de messages et réception

Dans cette partie, vous allez écrire une fonction permettant d'envoyer un message au LoRa Network Server.

Le LoRa Network Server (ou LNS) est configuré pour publier les messages reçus vers le topic : 
`TestTopic/lora/{appid}/{deveui}/{event}`  
A l'aide de mosquitto_sub, vous allez **subscribe** à ce topic depuis une console grâce à la commande suivante :  
`mosquitto_sub -h neocampus.univ-tlse3.fr -t TestTopic/lora/{appid}/{deveui}/# -p 1882 -u test -P test`

Le broker MQTT neOCampus utilise le port 1882. En général, les brokers MQTT utilisent le port 1883. On précise le broker grâce à l'argument `-p` dans la commande ci dessus. 
**Si vous essayez d'accéder au broker depuis un réseau externe à l'université, il faudra utiliser le port 10882**

In [17]:
def send_message(sp, message: bytes):
    """
    Send a message to the LoRa Network Server
    Param:
    sp: serial.Serial : serial.Serial object used for the RN2483
    message: str : Message to send
    Returns :
    response: str : Response from the module
    """
    # Message à envoyé
    command = f"mac tx uncnf 1 {message}"
    print(command)
    response = send(sp, command)
    return response

In [18]:
# Verification envoi de donnée
messages = "Hello, LoRaWAN BADER_NABI!"
message = bytes(messages,'utf-8')
print(message)
#response = send_message(module, message.hex())
#logging.debug("Response from server: %s", response)

b'Hello, LoRaWAN BADER_NABI!'


# 4. Application

Utilisez ce que vous avez appris dans ce TP pour réaliser une boucle pour envoyer des messages périodiquement via le module LoRa Mote.  
Attention ! Le time on air (ToA) est une denrée rare en LoRa. Réfléchissez et implémentez des mécanismes pour ne pas le gaspiller (tout en conservant une certaine périodicité dans l’envoi) 
Enfin, adaptez votre programme pour envoyer les données du SenseHat

Vous n'êtes pas obligés d'utiliser Jupyter Notebook pour votre programme.

In [19]:

from sense_hat import SenseHat
try:
    from cayennelpp import LppFrame
except ImportError:
    !pip install pycayennelpp
    from cayennelpp import LppFramemodule
def send_periodic_messages(sp, interval: int ):
    """
    Send periodic messages using the LoRa module
    Param:
    sp: serial.Serial : serial.Serial object used for the RN2483
    interval: int : Interval in seconds between messages
    """
    cpt = 0
    
    sense = SenseHat()
    while cpt < 30:
        lpp = LppFrame()
        
        # Lire les données du SenseHat
        temperature = sense.get_temperature()
        humidity = sense.get_humidity()
        pressure = sense.get_pressure()
        gyro = sense.get_gyroscope_raw()

        # Ajoute les données au LPP frame
        lpp.add_temperature(1, temperature)
        lpp.add_humidity(2, humidity) 
        lpp.add_pressure(3, pressure) 
        lpp.add_gyrometer(4, gyro['x'],gyro['y'],gyro['z'])
        
        message = lpp.to_bytes()
        # Envoi des données
        response = send_message(sp, message.hex())
        logging.info("Response from server: %s", response)
        # Wait for the next interval
        time.sleep(interval)
        cpt+=1
        print(cpt)
        
# Send messages every 30 secondes
send_periodic_messages(module, interval=30) 



DEBUG:PIL.PngImagePlugin:STREAM b'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM b'pHYs' 41 9
DEBUG:PIL.PngImagePlugin:STREAM b'tIME' 62 7
DEBUG:PIL.PngImagePlugin:b'tIME' 62 7 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM b'tEXt' 81 25
DEBUG:PIL.PngImagePlugin:STREAM b'IDAT' 118 774
DEBUG:root:Failed to initialise colour sensor. (Sensor not present)


mac tx uncnf 1 0167015f026832037327920486fff800030000


DEBUG:root:Decoded response : ok
INFO:root:Response from server: ok


KeyboardInterrupt: 